# Yelp API - Lab



## Introduction 

Now that we've seen how the Yelp API works, it's time to put those API and SQL skills to work in order to do some basic business analysis! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to:
* Create a DB on AWS to store information from Yelp about businesses
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and insert the information into your DB
* Perform pagination to retrieve troves of data!
* Write SQL queries to answer questions about your data 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner and combine it with our other data skills. In this lab you will get data from the Yelp API, store that data in a SQL Database on AWS, and write queries to answer follow-up questions. 


### Outline:

1. Determine which pieces of information you need to pull from the Yelp API.

2. Create a DB schema with 2 tables. One for the businesses and one for the reviews.

3. Create Python functions to:
  - Perform a search of businesses using pagination
  - Parse the API response for specific data points
  - Insert the data into your AWS DB

4. Use the functions above in a loop that will paginate over the results to retrieve all of the results. 

*Something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data*

5. Create functions to:
  - Retrieve the reviews data of one business
  - Parse the reviews response for specific review data
  - Insert the review data into the DB

6. Using SQL, query all of the business IDs. Using the 3 Python functions you've created, run your business IDs through a loop to get the reviews for each business and insert them into your DB.

7. Write SQL queries to answer the following questions about your data.


Bonus Steps:  
- Place your helper functions in a package so that your final notebook only has the major steps listed.
- Rewrite your business search functions to be able take an argument for the type of business you are searching for.
- Add another group of businesses to your database.


 
## SQL Questions:

- What are the 5 businesses with the highest average ratings?
- What are the 5 businesses with the lowest average ratings?
- What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs? 
- How many businesses have a rating greater than or  4.5?
- How many businesses have a rating less than 3?
- Return the text of the reviews for the most reviewed restaurant. 
- Return the name of the business with the most recent review. 
- Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the restaurant with the most reviews. 
- Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the restaurant with the least reviews. 


## Part I - Set up the DB

Start by reading SQL questions above to get an understanding of the data you will need. Then, read the documentation of Yelp API to understand what data you will receive in the response.  


Now that you are familiar with the data, create your SQL queries to create the DB and the appropriate tables. 

In [1]:
## Connect to DB server on AWS
import mysql.connector
from mysql.connector import errorcode

In [2]:
## Create new DB 
db_name = 'Yelp_Reviews'

In [3]:
import config

In [4]:
config.host

'kyledb.cyfsqcs7xvaz.us-east-2.rds.amazonaws.com'

In [5]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.passwd
)

print(cnx)

In [6]:
cur = cnx.cursor()

In [7]:
def create_database(cur, database):
    try:
        cur.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cur.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cur, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

In [58]:
# Create a table for the Businesses
db_name = 'Yelp_Reviews'

In [70]:
tables = {}
tables['businesses'] =(
     """CREATE TABLE businesses (
        id char(100) NOT NULL,
        name char(50) NOT NULL,
        city char(50),
        rating float(10),
        price char(10),
        reviews_count int(100),
        zip int(10),
        phone char(50),
        PRIMARY KEY (id))"""
)

In [71]:
tables['businesses']

'CREATE TABLE businesses (\n        id char(100) NOT NULL,\n        name char(50) NOT NULL,\n        city char(50),\n        rating float(10),\n        price char(10),\n        reviews_count int(100),\n        zip int(10),\n        phone char(50),\n        PRIMARY KEY (id))'

In [72]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.passwd,
    database = db_name
)
cur = cnx.cursor()

In [73]:
table_description = tables['businesses']
cur.execute(table_description)

In [ ]:
# Create a table for the reviews



## Part 2: Create ETL pipeline for the business data from the API

In [8]:
# write a function to make a call to the DB
import json
import requests

In [9]:
client_id = "9J_ck2JO3fi6RH5JLu8A-A"
api_key = "viVSrhO9Q0gaHiDJCaGZMUlypHjZRjgi5D6DIcQbcjxpkBbHncF1QEI48CNtGPiKpCoCO06tp6SnnzzWUMXwh6Sz-mFRT1Z0a5iD3L0n-D57m2hwBXxaNJOwZHh6XXYx"


In [10]:
term = 'Pizza'
location = 'New York City'
SEARCH_LIMIT = 10

url = 'https://api.yelp.com/v3/businesses/search' #endpoint 

headers = {
        'Authorization': 'Bearer {}'.format(api_key), # need to pass along the API key, this is how it asks for authorization
    }
#when you make the request, it will look into the header for this information 

url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': SEARCH_LIMIT
            }
response = requests.get(url, headers=headers, params=url_params)
print(response)
print(type(response.text))
print(response.text[:500])

<Response [200]>
<class 'str'>
{"businesses": [{"id": "zj8Lq1T8KIC5zwFief15jg", "alias": "prince-street-pizza-new-york-2", "name": "Prince Street Pizza", "image_url": "https://s3-media3.fl.yelpcdn.com/bphoto/ZAukOyv530w4KjOHC5YY1w/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/prince-street-pizza-new-york-2?adjust_creative=9J_ck2JO3fi6RH5JLu8A-A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=9J_ck2JO3fi6RH5JLu8A-A", "review_count": 3083, "categories": [{"alias": "pizza", "title": "Pizza"},


In [11]:
print(response.status_code)

200


In [12]:
response.json().keys()

dict_keys(['businesses', 'total', 'region'])

In [13]:
for key in response.json().keys():
    print(key)
    value = response.json()[key] #Use standard dictionary formatting
    print(type(value)) #What type is it?
    print('\n\n')

businesses
<class 'list'>



total
<class 'int'>



region
<class 'dict'>





In [14]:
yelp_pizza_data =response.json()

In [15]:
yelp_pizza_data

{'businesses': [{'id': 'zj8Lq1T8KIC5zwFief15jg',
   'alias': 'prince-street-pizza-new-york-2',
   'name': 'Prince Street Pizza',
   'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/ZAukOyv530w4KjOHC5YY1w/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/prince-street-pizza-new-york-2?adjust_creative=9J_ck2JO3fi6RH5JLu8A-A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=9J_ck2JO3fi6RH5JLu8A-A',
   'review_count': 3083,
   'categories': [{'alias': 'pizza', 'title': 'Pizza'},
    {'alias': 'italian', 'title': 'Italian'}],
   'rating': 4.5,
   'coordinates': {'latitude': 40.7229087705545,
    'longitude': -73.9944863319397},
   'transactions': ['delivery', 'pickup'],
   'price': '$',
   'location': {'address1': '27 Prince St',
    'address2': None,
    'address3': '',
    'city': 'New York',
    'zip_code': '10012',
    'country': 'US',
    'state': 'NY',
    'display_address': ['27 Prince St', 'New York, NY 10012']},
   'phone': '+12129664100',


In [16]:
yelp_pizza_data['businesses']

[{'id': 'zj8Lq1T8KIC5zwFief15jg',
  'alias': 'prince-street-pizza-new-york-2',
  'name': 'Prince Street Pizza',
  'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/ZAukOyv530w4KjOHC5YY1w/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/prince-street-pizza-new-york-2?adjust_creative=9J_ck2JO3fi6RH5JLu8A-A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=9J_ck2JO3fi6RH5JLu8A-A',
  'review_count': 3083,
  'categories': [{'alias': 'pizza', 'title': 'Pizza'},
   {'alias': 'italian', 'title': 'Italian'}],
  'rating': 4.5,
  'coordinates': {'latitude': 40.7229087705545,
   'longitude': -73.9944863319397},
  'transactions': ['delivery', 'pickup'],
  'price': '$',
  'location': {'address1': '27 Prince St',
   'address2': None,
   'address3': '',
   'city': 'New York',
   'zip_code': '10012',
   'country': 'US',
   'state': 'NY',
   'display_address': ['27 Prince St', 'New York, NY 10012']},
  'phone': '+12129664100',
  'display_phone': '(212) 966-4100',


In [17]:
dict(response.headers)

{'Connection': 'keep-alive',
 'Server': 'nginx',
 'Content-Type': 'application/json',
 'RateLimit-DailyLimit': '5000',
 'RateLimit-Remaining': '5000',
 'RateLimit-ResetTime': '2019-09-14T00:00:00+00:00',
 'X-Zipkin-Id': '6f4aa3ff4b85883e',
 'X-B3-Sampled': '0',
 'X-Proxied': '10-65-175-107-useast1bprod',
 'Content-Encoding': 'gzip',
 'Accept-Ranges': 'bytes',
 'Date': 'Fri, 13 Sep 2019 13:31:54 GMT',
 'Via': '1.1 varnish',
 'X-Served-By': 'cache-lga21938-LGA',
 'X-Cache': 'MISS',
 'X-Cache-Hits': '0',
 'transfer-encoding': 'chunked'}

In [18]:
# write a function to parse the API response 
# so that you can easily insert the data in to the DB

restaurant_info= {}    
for restaurant in yelp_pizza_data['businesses']:
    restaurant_info['id']= restaurant['id']
    restaurant_info['name']= restaurant['name']
    for i in restaurant['location']:
        restaurant_info['city']= restaurant['location']['city']
        restaurant_info['zip']= restaurant['location']['zip_code']
    restaurant_info['reviews_count']= restaurant['review_count']
    restaurant_info['phone']= restaurant['phone']

    
    
    

    
#      """CREATE TABLE businesses (


#     city char(50),
#         rating float(10),
#         price char(10),
#         reviews_count int(100),
#         zip int(10),
#         phone char(50),
#         PRIMARY KEY (id))"""
restaurant_info

{'id': 'nUpA3sY5b6dynC7aj0eV6A',
 'name': "Rizzo's Fine Pizza",
 'city': 'New York',
 'zip': '10002',
 'reviews_count': 708,
 'phone': '+16464541262'}

In [19]:
restaurant = yelp_pizza_data['businesses'][0]

In [20]:
def dict_maker(dictionary):   
    
    
    restaurant_info= {}    
    for restaurant in dictionary:
        restaurant_info['id']= restaurant['id']
        restaurant_info['name']= restaurant['name']
        for i in restaurant['location']:
            restaurant_info['city']= restaurant['location']['city']
            restaurant_info['zip']= restaurant['location']['zip_code']
        restaurant_info['reviews_count']= restaurant['review_count']
        restaurant_info['phone']= restaurant['phone']
        return restaurant_info 

In [22]:
for business in yelp_pizza_data['businesses']:
    dict_maker(yelp_pizza_data)

TypeError: string indices must be integers

In [84]:
restaurant_info

{'id': 'ysqgdbSrezXgVwER2kQWKA',
 'name': "Juliana's Pizza",
 'city': 'Brooklyn',
 'zip': '11201',
 'reviews_count': 2071,
 'phone': '+17185966700'}

In [ ]:
# Write a function to take your parsed data and insert it into the DB

## Part 3: Create ETL pipeline for the restaurant review data from the API

In [ ]:
# write a query to pull back all of the business ids 
# you will need these ids to pull back the reviews for each restaurant

In [ ]:
# write a function that takes a business id 
# and makes a call to the API for reivews
# then parse out the relevant information

In [ ]:
# write a function to insert the parsed data into the reviews table

## Part 4: Write SQL queries that will answer the questions posed. 

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter.***

In [ ]:
# Your code here; use a function or loop to retrieve all the results from your original request
import time

def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
    data = response.json()['businesses']
    return data

def all_results(url_params, api_key):
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    results = []
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        results.append(yelp_call(url_params, api_key))
        time.sleep(1) #Wait a second
        results += 50
    return df

term = 'pizza'
location = 'Astoria NY'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }
df = all_results(url_params, api_key)
print(len(df))
df.head()

### Sample SQL Query 

Below is a SQL query to create a table.  Additionally here is a link to create a table with a foreign key.

http://www.mysqltutorial.org/mysql-foreign-key/

```CREATE TABLE IF NOT EXISTS tasks (
    task_id INT AUTO_INCREMENT,
    title VARCHAR(255) NOT NULL,
    start_date DATE,
    due_date DATE,
    status TINYINT NOT NULL,
    priority TINYINT NOT NULL,
    description TEXT,
    PRIMARY KEY (task_id)
)  ENGINE=INNODB;```